# T-725 Natural Language Processing: Lab 7
In today's lab, we will be working with spaCy and Huggingface for a variety of tasks. We'll also learn how to use Gradio to quickly create convenient user interfaces.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* **Select `"Runtime" > "Change runtime type"`, and make sure that you have "Hardware accelerator" set to "GPU"**
* Select `"Runtime" > "Run all"` to run the code in this notebook.

## spaCy

[spaCy](https://spacy.io) is a free, open-source library for advanced Natural Language Processing (NLP) in Python.

spaCy is designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning.

### Features

Name | Description
---|---
**Tokenization** | Segmenting text into words, punctuations marks etc.
**Part-of-speech (POS) Tagging** | Assigning word types to tokens, like verb or noun.
**Dependency Parsing** | Assigning syntactic dependency labels, describing the relations between individual tokens, like subject or object.
**Lemmatization** |	Assigning the base forms of words. For example, the lemma of “was” is “be”, and the lemma of “rats” is “rat”.
**Sentence Boundary Detection (SBD)** |	Finding and segmenting individual sentences.
**Named Entity Recognition (NER)** | Labelling named “real-world” objects, like persons, companies or locations.
**Entity Linking (EL)** | Disambiguating textual entities to unique identifiers in a knowledge base.
**Similarity** | Comparing words, text spans and documents and how similar they are to each other.
**Text Classification** | Assigning categories or labels to a whole document, or parts of a document.
**Rule-based Matching** | Finding sequences of tokens based on their texts and linguistic annotations, similar to regular expressions.
**Training** | Updating and improving a statistical model’s predictions.
**Serialization** | Saving objects to files or byte strings.

### Trained Pipelines

While some of spaCy’s features work independently, others require [trained pipelines](https://spacy.io/models) to be loaded, which enable spaCy to predict linguistic annotations – for example, whether a word is a verb or a noun. A trained pipeline can consist of multiple components that use a statistical model trained on labeled data. spaCy currently offers trained pipelines for a variety of languages, which can be installed as individual Python modules.

### Summarization Example
Let's take a look at some of the functionality of spaCy through the example of [automatic summarization](https://medium.com/luisfredgs/automatic-text-summarization-with-machine-learning-an-overview-68ded5717a25). There are two main types of summarization: extractive and abstractive. Extractive summarization selects a subset of sentences from the text to form a summary; abstractive summarization reorganizes the language in the text and adds novel words/phrases into the summary if necessary.

For this example we'll be doing automatic [extractive summarization](https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744).

First install spaCy:

In [100]:
!pip install -U spacy -q

Then import all necessary modules:

In [101]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

There are many [other languages](https://spacy.io/usage/models) to choose from. Here we load the English language models:

In [102]:
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Choose some text to be summarized and store it in a variable:

In [103]:
long_text = """
Machine learning (ML) is the scientific study of algorithms and statistical models
that computer systems use to progressively improve their performance on a specific
task. Machine learning algorithms build a mathematical model of sample data, known as
“training data”, in order to make predictions or decisions without being explicitly
programmed to perform the task. Machine learning algorithms are used in the applications
of email filtering, detection of network intruders, and computer vision, where it
is infeasible to develop an algorithm of specific instructions for performing the task.
Machine learning is closely related to computational statistics, which focuses on
making predictions using computers. The study of mathematical optimization delivers
methods, theory and application domains to the field of machine learning. Data mining
is a field of study within machine learning and focuses on exploratory data analysis
through unsupervised learning. In its application across business problems, machine
learning is also referred to as predictive analytics.
"""

Pass the text to the `nlp` function:

In [104]:
doc = nlp(long_text)

At this point, the text has been processed, i.e., tokenized, lemmatized, tagged with parts-of-speech, and parsed. A variety of lingustic features are accessbile via the `doc` object, e.g.:

* Lemmas
* Parts of speech
* Dependency parse
* Named entities
* Chunks
* Is alphabet character
* Is capitalized
* Is in the stop list

The following will print out each of those bits of information for every token in the original text, one token per line:

In [105]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


 
 SPACE _SP dep 
 False False
Machine Machine PROPN NNP compound Xxxxx True False
learning learning NOUN NN nsubj xxxx True False
( ( PUNCT -LRB- punct ( False False
ML ML PROPN NNP appos XX True False
) ) PUNCT -RRB- punct ) False False
is be AUX VBZ ROOT xx True True
the the DET DT det xxx True True
scientific scientific ADJ JJ amod xxxx True False
study study NOUN NN attr xxxx True False
of of ADP IN prep xx True True
algorithms algorithm NOUN NNS pobj xxxx True False
and and CCONJ CC cc xxx True True
statistical statistical ADJ JJ amod xxxx True False
models model NOUN NNS conj xxxx True False

 
 SPACE _SP dep 
 False False
that that SCONJ IN mark xxxx True True
computer computer NOUN NN compound xxxx True False
systems system NOUN NNS nsubj xxxx True False
use use VERB VBP relcl xxx True False
to to PART TO aux xx True True
progressively progressively ADV RB advmod xxxx True False
improve improve VERB VB xcomp xxxx True False
their their PRON PRP$ poss xxxx True True
performanc

Next, we'll use this information to filter keywords from the original text.

* Define the keywords list
* Choose the parts-of-speech that are likely to be important ([pos tags in spaCy](https://spacy.io/usage/linguistic-features/#pos-tagging))
* Skip tokens that are in the stop list
* Add tokens that have the part-of-speech we care about to the keywords list

In [106]:
keywords = []
pos_tags = ["PROPN", "ADJ", "NOUN", "VERB"]
for token in doc:
    if token.is_stop:
        continue
    if token.pos_ in pos_tags:
        keywords.append(token.text)

Next, we calculate the frequency of each token using the `Counter` function and store it in `freq_words`.

To view the top `n` most frequent words, the `most_common(n)` method can be used:

In [107]:
freq_words = Counter(keywords)
freq_words.most_common(5)

[('learning', 8), ('Machine', 4), ('study', 3), ('algorithms', 3), ('task', 3)]

This frequency should be normalised for better processing and it can be done by dividing the token's frequencies by the maximum frequency:

In [108]:
max_freq = freq_words.most_common(1)[0][1] # most_common(1) returns a 1 element list, containing a tuple where the frequency is the 2nd element
for word in freq_words.keys():
    freq_words[word] = (freq_words[word]/max_freq)

freq_words.most_common(5)

[('learning', 1.0),
 ('Machine', 0.5),
 ('study', 0.375),
 ('algorithms', 0.375),
 ('task', 0.375)]

Next, we weigh each sentence based on the frequency of the keyword token present in each sentence. The result is stored as a key-value pair in `sent_strength` where keys are the sentences and the values are the weight of each sentence:

In [109]:
sent_strength = {}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_words.keys():
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_words[word.text]
            else:
                sent_strength[sent] = freq_words[word.text]

print(sent_strength)

{
Machine learning (ML) is the scientific study of algorithms and statistical models
that computer systems use to progressively improve their performance on a specific
task.: 4.125, Machine learning algorithms build a mathematical model of sample data, known as
“training data”, in order to make predictions or decisions without being explicitly
programmed to perform the task.: 4.625, Machine learning algorithms are used in the applications
of email filtering, detection of network intruders, and computer vision, where it
is infeasible to develop an algorithm of specific instructions for performing the task.
: 4.25, Machine learning is closely related to computational statistics, which focuses on
making predictions using computers.: 2.625, The study of mathematical optimization delivers
methods, theory and application domains to the field of machine learning.: 3.125, Data mining
is a field of study within machine learning and focuses on exploratory data analysis
through unsupervised learn

Next, the `nlargest` function is used to summarize the string. It takes 3 arguments:

* Number of elements to extract
* An Iterable (List/Tuple/Dictionary)
* Condition to be satisfied

This nlargest function returns a list containing the 3 sentences with the highest sentence strength score calculated in the previous step.

We store this output in `summarized_sentences`:

In [110]:
summarized_sentences = nlargest(4, sent_strength, key=sent_strength.get)

print(summarized_sentences)

[Machine learning algorithms build a mathematical model of sample data, known as
“training data”, in order to make predictions or decisions without being explicitly
programmed to perform the task., Machine learning algorithms are used in the applications
of email filtering, detection of network intruders, and computer vision, where it
is infeasible to develop an algorithm of specific instructions for performing the task.
, Data mining
is a field of study within machine learning and focuses on exploratory data analysis
through unsupervised learning., 
Machine learning (ML) is the scientific study of algorithms and statistical models
that computer systems use to progressively improve their performance on a specific
task.]


Lastly, convert the text data in the `summarized_sentences` to a string and print it:

In [111]:
final_sentences = [w.text for w in summarized_sentences]
summary = ' '.join(final_sentences)
print(summary)

Machine learning algorithms build a mathematical model of sample data, known as
“training data”, in order to make predictions or decisions without being explicitly
programmed to perform the task. Machine learning algorithms are used in the applications
of email filtering, detection of network intruders, and computer vision, where it
is infeasible to develop an algorithm of specific instructions for performing the task.
 Data mining
is a field of study within machine learning and focuses on exploratory data analysis
through unsupervised learning. 
Machine learning (ML) is the scientific study of algorithms and statistical models
that computer systems use to progressively improve their performance on a specific
task.


This example only shows a very limited application of [spaCy](https://spacy.io). The package has many powerful tools to create NLP applications.

##Gradio
[Gradio](https://gradio.app) is a fast way to demo your machine learning model with a nice web interface so that anyone can use it. The possibilities with Gradio are vast, this lab only scratches the surface.

Here's the setup for a very basic UI:

* First, define a function that does your main processing when users click the 'Submit' button in the UI.
* Then define a gradio `Interface` called `demo`. This constructor has several arguments:
  * The first in this example is the name of the function you defined
  * The second is the type of inputs you want to capture (one text input in this case)
  * The third is the type of output (also text)
* Lastly, call the `Interface` object's `launch()` function to render the UI.

In [112]:
!pip install gradio -q

In [113]:
import gradio as gr

In [114]:
def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ed5c81765837343c46.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Sumy

You can do any kind of processing inside your main function, like call other functions. Let's create a quick summarization tool using [sumy](https://github.com/miso-belica/sumy).

* First, install the necessary packages for sumy.
* Then import the modules for the summarization task.

In [115]:
!pip install sumy -q

In [116]:
import nltk
nltk.download('punkt', quiet=True)

True

In [117]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

LANGUAGE = "english"

def sumy_summarize(txt, n_sent=1):
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    parser = PlaintextParser.from_string(txt, Tokenizer(LANGUAGE))
    sents = ""
    for sentence in summarizer(parser.document, n_sent):
        sents += str(sentence) + "\n"

    return sents

Return the output of the `sumy_summarize` function inside your main Gradio function, passing it the user input.

Try using the text from the previous summarization example as input and experiment with the `n_sent` parameter.

In [118]:
def sum(text):
    return sumy_summarize(text)

sum_demo = gr.Interface(fn=sum, inputs="text", outputs="text")
sum_demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ff33e8f7fc3e12ddf6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The `sumy_summarize` function takes another parameter in addition to the text:

* n_sent (int or None, optional) – The number of sentences of the original text to be chosen for the summary.

Let's add more input elements to the interface:

* A checkbox widget allowing the user to capitalize the output
* A text box to set the `n_sent`

In [119]:
def sum(text, make_caps, number_of_sentences):
    summary = sumy_summarize(text, n_sent=number_of_sentences)
    return summary.upper() if make_caps else summary

# The input objects are used in sequence as arguments to the fn function
sum_demo = gr.Interface(
    fn=sum,
    inputs=["text", "checkbox", "number"],
    outputs="text"
)

sum_demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c0a92dc379229a3998.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The `Interface` is highly customizable. For example, you can give each element different labels, have placeholder text, change colors, etc. Check the [Gradio documentation](https://gradio.app/docs/) for details.

## Hugging Face


🤗 [Transformers](https://huggingface.co/docs/transformers/index) is a state-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX, and provides APIs and tools to easily download and train state-of-the-art pretrained models.

Begin by installing the Hugging Face `transformers` library:

In [120]:
!pip install transformers -q

Import the necessary modules for the tasks up ahead:

In [121]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, Conversation

Hugging Face pipelines make it simple to use any model for inference on any language, computer vision, speech, and multimodal tasks (see [docs](https://huggingface.co/docs/transformers/pipeline_tutorial)). The `pipeline()` automatically loads a default model and a preprocessing class capable of inference for your task.

The following is an example of using a Hugging Face pipeline to do automatic abstractive summarization:

* First create a pipeline object, here called `summarizer`
* The `pipeline()` constructor takes two arguments:
  * The name of the task (see [docs for existing pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)) - In our case, the task is 'summarization'
  * The model - We can use [facebook/bart-large-xsum](https://huggingface.co/facebook/bart-large-xsum). This can be changed to any number of models available for this task on Hugging Face.

* Next we define a string variable containing the text we want to summarize. Here it's called `text_to_summarize`.
* Then call the `summarizer` pipeline, pass the string, and optionally set the max and min length, and method for generation
* Finally, print the output

In [122]:
summarizer = pipeline("summarization", model="facebook/bart-large-xsum")

In [123]:
text_to_summarize = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [124]:
summary = summarizer(text_to_summarize, max_length=130, min_length=30, do_sample=False)
print(summary[0]["summary_text"])

A New York woman has pleaded not guilty to falsely claiming to be married 10 times, including to eight men from different countries, in what prosecutors say was an immigration scam.


Let's take things up a notch and create a non-goal-oriented chatbot using pipelines and Gradio.

We can define our model and tokenizer explicitly to choose the model best suited for the task, instead of relying on the pipeline's default model. A good model for aimless chit-chat is the [Blenderbot](https://huggingface.co/facebook/blenderbot-400M-distill) model. Note that there are many other chat models you could choose for this task.

Start by defining the tokenizer and model:

In [125]:
chat_tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
chat_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Next, we create a 'conversational' pipeline called `bot` using the model and tokenizer we defined above:

In [126]:
bot = pipeline(task="conversational", model=chat_model, tokenizer=chat_tokenizer)

In order to build a basic chatbot interface using a pretrained model, we take the following steps:

* Initialize the `Conversation` object outside the function, in order to keep building the dialog history over time, so the model has the dialog context when it generates responses.
* Define a `chat` function that takes text from user input and keeps a history of the dialog across turns. This function:
  * Adds the user's current input to the `Conversation` by calling `add_user_input` on the `convo` object, passing the `input`.
  * Passes the `convo` object to the `bot` to make the model generate a response.
  * The `bot` object outputs the whole conversation by default, so we call `generated_responses` to get a simple list of the generated responses so far, and get the last item in the list (i.e., the most recent response).
  * Then we update the `history`, which is a list of tuples: (user-input, bot-response)
* Lastly, we define a Gradio `Interface` that:
  * Calls the `chat` function
  * Takes text as input
  * Creates widgets from a predefined Gradio layout for a 'chatbot'
  * Retains a 'state' of the dialog by retaining a list of the chat `history`.


In [127]:
convo = Conversation()
def chat(input, history=[]):
    convo.add_user_input(input)
    output = bot([convo]).generated_responses[-1]
    history.append(input)
    history.append(output)
    response = [(history[i], history[i+1]) for i in range(0, len(history)-1, 2)]
    return response, history

interface = gr.Interface(
    fn=chat,
    theme="default",
    css=".footer {display:none !important}",
    inputs=["text", "state"],
    outputs=["chatbot", "state"],
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b4f9ba6751b75896d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


You now have a variety of powerful tools at your disposal to rapidly create interesting and useful NLP applications!

# Assignment
Complete the following questions and hand in your solution in Canvas before 8:30 Monday morning, October 16th. Remember to save your file before uploading it.

## Part 1
Compare the outputs of the three summarization methods covered in this notebook.

Use the same piece of text for each method to answer these questions:

1. Which method performs the best, in your opinion?
2. What are the pros and cons of each method?
3. What kind of summarization is each method doing?

In [128]:
# Your solution here
declaration = """
The unanimous Declaration of the thirteen united States of America, When in the Course of human events,
it becomes necessary for one people to dissolve the political bands which have connected them with another,
and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature
and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should
declare the causes which impel them to the separation.
We hold these truths to be self-evident, that all men are created equal, that they are endowed by their
Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.
That to secure these rights, Governments are instituted among Men, deriving their just powers from the
consent of the governed. That whenever any Form of Government becomes destructive of these ends, it is
the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation
on such principles and organizing its powers in such form, as to them shall seem most likely to effect
their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be
changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more
disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which
they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object
evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off
such Government, and to provide new Guards for their future security. Such has been the patient sufferance
of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government.
The history of the present King of Great Britain is a history of repeated injuries and usurpations, all
having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts
be submitted to a candid world.
"""

In [129]:
def spacy_summary(text, sents_count=4, language='en_core_web_sm', pos_tags = ["PROPN", "ADJ", "NOUN", "VERB"]):
    nlp = spacy.load(language)
    doc = nlp(text)

    keywords = []
    pos_tags = ["PROPN", "ADJ", "NOUN", "VERB"]
    for token in doc:
        if token.is_stop:
            continue
        if token.pos_ in pos_tags:
            keywords.append(token.text)

    freq_words = Counter(keywords)
    max_freq = freq_words.most_common(1)[0][1]

    for word in freq_words.keys():
        freq_words[word] = (freq_words[word]/max_freq)

    sent_strength = {}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_words.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent] += freq_words[word.text]
                else:
                    sent_strength[sent] = freq_words[word.text]

    summarized_sentences = nlargest(sents_count, sent_strength, key=sent_strength.get)

    final_sentences = [w.text for w in summarized_sentences]
    summary = ' '.join(final_sentences)

    return summary

In [130]:
print(spacy_summary(declaration, sents_count=2))

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



The unanimous Declaration of the thirteen united States of America, When in the Course of human events,
it becomes necessary for one people to dissolve the political bands which have connected them with another,
and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature
and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should
declare the causes which impel them to the separation.
 That whenever any Form of Government becomes destructive of these ends, it is
the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation
on such principles and organizing its powers in such form, as to them shall seem most likely to effect
their Safety and Happiness.


In [131]:
def sumy_summary(txt, sents_count=1, language="english"):
    stemmer = Stemmer(language)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(language)

    parser = PlaintextParser.from_string(txt, Tokenizer(language))
    sents = ""
    for sentence in summarizer(parser.document, sents_count):
        sents += str(sentence) + "\n"

    return sents

In [132]:
print(sumy_summary(declaration, sents_count=2))

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.
But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.



In [133]:
def transformer_summary(txt, min_len=30, max_len=130, model="facebook/bart-large-xsum"):
    summarizer = pipeline("summarization", model=model)
    summary = summarizer(txt, min_length=min_len, max_length=max_len, do_sample=False)

    return summary[0]["summary_text"]

In [134]:
print(transformer_summary(declaration, max_len=400))

Here is the text of the Declaration of Independence of the United States of America, signed by all the states of America at the same time, on July 4, 1776.


I'd say the best performance came from the HuggingFace summarizer, which actually produced an original text that broadly describes the input, while the others directly copied sentences in their entirety (although those where indeed key concept of the original text).

## Part 2
Create a sentiment classifier using Gradio and Huggingface.

* Augment the simple version of the Gradio interface.
* Add a Huggingface 'text-classification' pipleline, using the 'cardiffnlp/twitter-roberta-base-sentiment' model.

This model outputs a list that contains one dictionary object. In this dictionary, the predicted class is the value of the key 'label'. The model outputs one of three sentiment classes:

* `LABEL_0` for negative
* `LABEL_1` for neutral
* `LABEL_2` for positive

Your app should take some text as input and output **one** of these three words:

* Positive
* Neutral
* Negative

In [135]:
# Alter this codeblock and/or create as many blocks as necessary to accomplish the task for this part
classif = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

classif(declaration)[0]

{'label': 'LABEL_1', 'score': 0.7346365451812744}

In [136]:
def analyze(text):
    res = list(classif(text)[0].values())

    label_idx = res[0][-1]
    labels = {
        "0": "positive",
        "1": "neutral",
        "2": "negative"
    }

    return labels[label_idx], res[1]

sentiment_analyzer = gr.Interface(
    fn=analyze,
    inputs=gr.Textbox(label="Input text"),
    outputs=[gr.Textbox(label="Sentiment label"), gr.Textbox(label="Score")]
)

sentiment_analyzer.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://41030362b15c496e76.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Part 3

Create a chatbot that can do these three things:

*   Summarize text
*   Analyze sentiment
*   Mindless chit-chat

Augment the chatbot code below to accomplish the following:

1. When the user ticks a checkbox that says 'do_summary':
  * The user's long text input should be summarized by the bot using the pipeline defined earlier and responds with the summary
  * This process may bypass the default chit-chat functionality
2. When the user ticks a checkbox that says 'do_sentiment':
  * The user's text input may be analyzed by the bot for sentiment using the method created in Part 2 and responds accordingly
  * This process may bypass the default chit-chat functionality
3. Process any other input using the default chat functionality

In [140]:
# Alter this codeblock and/or create as many blocks as necessary to accomplish the task for this part

bot = pipeline(task="conversational", model=chat_model, tokenizer=chat_tokenizer)

convo = Conversation()
def chat(input, history=[]):
    convo.add_user_input(input)
    output = bot([convo]).generated_responses[-1]
    history.append(input)
    history.append(output)
    response = [(history[i], history[i+1]) for i in range(0, len(history)-1, 2)]
    return response, history

def main(input, radio):
    if radio == "Sentiment":
        return analyze(input)[0]
    elif radio == "Summary":
        return transformer_summary(input)

    return chat(input)[0]


interface = gr.Interface(
    fn=main,
    theme="default",
    css=".footer {display:none !important}",
    inputs=["text", gr.Radio(
        ["Chat", "Summary", "Sentiment"], label="What kind of essay would you like to write?", value="Chat"
    )],
    outputs=["chatbot"],
)

interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fd13b94b199a13633c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Your max_length is set to 130, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1563, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1451, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 262, in postprocess
  

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7891 <> https://fd13b94b199a13633c.gradio.live
